In [ ]:
library(DBI)
library(dplyr)
library(stats)
library(devtools)
install_github("karpatit/mechkar",force = TRUE)
install.packages('colorspace')
library(ggplot2)
library(colorspace)

In [ ]:
movies <- read.csv("/Users/maozdotan/Documents/DS/SQL/BoxOffice_EDA_ff.csv") 
head(movies)

In [ ]:
movies$revenue <- as.numeric(movies$revenue)
movies$runtime <- as.numeric(movies$runtime)
movies$actors_male_count <- as.numeric(movies$actors_male_count)
movies$actors_female_count <- as.numeric(movies$actors_female_count)
movies$Avg_Popularity_5Y_Actor0 <- as.numeric(movies$Avg_Popularity_5Y_Actor0)
movies$Avg_Popularity_5Y_Actor1 <- as.numeric(movies$Avg_Popularity_5Y_Actor1)
movies$popularity_cat <- factor(movies$popularity_cat)
movies$runtime_cat <- factor(movies$runtime_cat)
#movies$runtime_cat <- factor(movies$runtime_cat,levels=c(1,2,3),labels=c("Short","medium","long"))
movies$homepage <- factor(movies$homepage)
movies$poster_path <- factor(movies$poster_path)
movies$tagline <- factor(movies$tagline)
movies$in_collection <- factor(movies$in_collection)
movies$original_lang_en <- factor(movies$original_lang_en)
movies$top_genere <- factor(movies$top_genere)
movies$Directing <- as.numeric(movies$Directing)
movies$Art <- as.numeric(movies$Art)
movies$Sound <- as.numeric(movies$Sound)
movies$Crew <- as.numeric(movies$Crew)
movies$Costume...Make.Up <- as.numeric(movies$Costume...Make.Up)
movies$Crew <- as.numeric(movies$Crew)
movies$Production <- as.numeric(movies$Production)
movies$Camera <- as.numeric(movies$Camera)
movies$Lighting <- as.numeric(movies$Lighting)
movies$Editing <- as.numeric(movies$Editing)
movies$Writing <- as.numeric(movies$Writing)
head(movies)

In [ ]:
col <- colnames(movies)
col

# חלק 2

In [ ]:
summary(movies)

In [ ]:
### characters
strlst <- NULL
numlst <- NULL
for (v in names(movies)) {
    if(typeof(movies[[v]])=='character' | is.factor(movies[[v]])) {
        strlst <- c(strlst,v) 
    } else {
        if (v != 'revenue') 
            numlst <- c(numlst, v)
    }
}

for (v in strlst) {
    if(typeof(movies[[v]])=='character') {
        movies[[v]] <- factor(movies[[v]])
    }
}
strlst
numlst

In [ ]:
tab1 <- suppressWarnings(mechkar::Table1(data=movies[,3:ncol(movies)]))

In [ ]:
tab1

In [ ]:
suppressWarnings(mechkar::exploreData(data=movies,y="revenue"))

In [ ]:
tab1[tab1$Variables == 'top_genere',]

In [ ]:
for (n in col[3:length(col)]) {
    print (n)
    ifelse(n %in% c('release_year','release_month','revenue'),print(''),
        print (tab1[tab1$Variables == n,])
           )}

In [ ]:
table(movies$keyword_cnt)

In [ ]:
# Count unique values
sapply(movies, function(x) length(unique(x)))

In [ ]:
# Find how many errors in year value
length(which(movies$year > 2017))

In [ ]:
movies <- movies %>% mutate(release_year=as.numeric(movies$release_year))
movies <- movies %>% mutate(release_year= ifelse(release_year>2020,release_year-100,release_year))
c(min(movies$release_year,na.rm=T),max(movies$release_year,na.rm=T))

שאלה 2.2
גרפים המתארים את ההתנהגות של כל משתנה

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
for (n in names(movies[3:length(movies)])) {
    suppressWarnings(
        if (n %in% numlst) {hist(movies[[n]],xlab=n, main=n)}
    else {plot(movies[[n]],main=n)})
}

# EDA

שאלה 2.3 מטריצת קורלציות

In [ ]:
library(MESS)
####
options(repr.plot.width = 20, repr.plot.height = 20)
pairs(movies[,numlst], 
      upper.panel = panel.smooth,
      diag.panel = panel.hist, 
      lower.panel = NULL,
      cex=0.8)



In [ ]:
options(repr.plot.width = 25, repr.plot.height = 25)
library(corrgram)
dfcormat <- cor(movies[,numlst], use = "pairwise.complete.obs", method = "spearman")
corrgram::corrgram(dfcormat)

In [ ]:
library(rcompanion)
library(lsr)
res <- correlate(movies, corr.method = "spearman")
res

In [ ]:
df1 <- data.frame()
df1 <- NULL
for (m in c('popularity_cat','runtime_cat','runtime_cat','tagline','original_lang_en','top_genere')) {
            df$var1 <- "Popularity"
            df$var2 <- m
            df1$cramersV <- cramersV(movies$popularity_cat, movies[[m]])
        }
df1
cramersV(movies$popularity_cat, movies$runtime_cat)
cramersV(movies$popularity_cat, movies$original_lang_en)
cramersV(movies$popularity_cat, movies$homepage)
cramersV(movies$popularity_cat, movies$tagline)
cramersV(movies$popularity_cat, movies$top_genere)
cramersV(movies$popularity_cat, movies$in_collection)

שאלה 2.4
משתנה מטרה 
Revenue

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
summary(movies$revenue)
hist(movies$revenue, main='Revenue')
hist(log(movies$revenue+1) ,main='Revenue log')
plot(log(movies$budget+1) ~ log(movies$revenue+1), main='Revenue vs. Budget')
plot(log(movies$Avg_Popularity_5Y_Actor0+1) ~ log(movies$revenue+1), main='Revenue vs. Avg_Popularity_5Y_Actor0')
plot(log(movies$Avg_Popularity_5Y_Actor1+1) ~ log(movies$revenue+1),main='Revenue vs. Avg_Popularity_5Y_Actor1')

for (n in numlst){
    if(!n %in% c('budget','Avg_Popularity_5Y_Actor0','Avg_Popularity_5Y_Actor1')) {
        plot(log(movies$revenue+1) ~ movies[[n]], main= n)
    }
}

In [ ]:
#categoial variables
for (v in names(movies[,3:length(movies)])) { #do not use id and title for review
    if(is.factor(movies[[v]])==T) {
        suppressWarnings(boxplot(log(movies$revenue+1) ~ movies[[v]], main=v, outline=F))
    }
}

The outcome has high positive corraletion to :
1. budget
2. popularity

It is associated with:
1. average popularity of main actors (order 0 - positive up to popularity 40 and then negative)
2. female/male actor ratio - strong correlation up to ~0.5 and then no correlation
3. original language english
4. homepage available
5. tagline
6. top genre

variable correlation with:
1. keyword count - positive up to 50 and then negative
2. in collection - strong positive up to 7 and then negative
3. rutime - strong positive bteween 100-200 min and negative beyond 200 min

שאלה 2.5
זיהוי נתוני קיצון

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
for (n in names(movies[3:length(movies)])) {
    suppressWarnings(
        if (n %in% numlst) {boxplot(movies[[n]],xlab=n, main=n)})
}

The following variables have ouliers:

Popularity,budget,runtime,keyword_cnt,Actor_male_cnt,avg_popularity_5Y_Actor0, avg_popularity_5Y_Actor1, Crew

These vars have low number of outliers:

Actor_female_cnt,Writing,Directing,Art,Sound, Costume,Production,Camera,Lighting,Editing,

שאלה 2.6
זיהוי נתונים חסרים

In [ ]:
mm <- mechkar::getMissingness(data = movies)
mm$missingness
mm$missingness$var

שאלה 2.7
מטריצת נתונים חסרים

In [ ]:
missingMatrix <- function(data) {
    vn <- names(data)
    missdata <- data.frame(row1=1:nrow(data))
    for(v in vn) {
        mv <- ifelse(is.na(data[[v]]),1,0)
        missdata[v] <- mv
    }
    missdata$row1 <- NULL
    return(missdata)
}

In [ ]:
## generate a missing matrix
df_na <- missingMatrix(movies)

In [ ]:
#generate a missing matrix
library(naniar)
options(repr.plot.width = 20, repr.plot.height = 12)
vis_miss(movies,warn_large_data=F)

# Univariate Analysis

### Popularity

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(data=movies) +
    geom_density(aes(popularity))
ggplot(data=movies) +
    geom_density(aes(log(popularity+1)))
plot(movies$popularity)

In [ ]:
length(which(movies$popularity > 10.9))# values higher that Q3
length(which(movies$popularity > 25))
IQR(movies$popularity)
quantile(movies$popularity)

we see many ouliers above Q3 (1857 figures)
from the scatter plot it seems that we can set threshold for outliers if higher than 25

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
plot(runtime ~ popularity, data=movies)
plot(revenue_rank ~ popularity, data=movies)
plot(keyword_cnt ~ log(popularity)+1, data=movies)
plot(actors_female_count ~ log(popularity)+1, data=movies)
plot(actors_male_count ~ popularity, data=movies)
plot(actors_female_ratio ~ log(popularity)+1, data=movies)

In [ ]:
# explore popularity vs. female/male ratio 
ggplot(data=movies) +
    geom_point(aes(x=actors_female_ratio, y=sqrt(popularity)+0.1))

movies with ratio<1 (Less female actors) tend to achive higher popularity

In [ ]:
plot(log(movies$revenue+1) ~ log(movies$popularity+0.1))
plot(log(movies$budget+1) ~ log(movies$popularity+0.1))
plot(popularity ~ id, data = movies)
ggplot(data=movies) +
    geom_point(aes(x=id, y=popularity)) +
    geom_hline(yintercept = 50,color="red")
movies1 <- filter(movies,popularity < 50) # set threshold 50 to analize without ouliers
## show relations without outliers in Popularity
plot( data=movies1,popularity~revenue )
plot(popularity~sqrt(budget) , data=movies1)
plot(popularity~keyword_cnt, data=movies1)

Found relationship between Popularity and revenue,budget

In [ ]:
### analize categorial variables
boxplot(popularity ~ popularity_cat, data = movies)
boxplot(popularity ~ runtime_cat, data = movies)
boxplot(popularity ~ homepage, data = movies)
boxplot(popularity ~ tagline, data = movies)
boxplot(popularity ~ in_collection, data = movies)
boxplot(popularity ~ original_lang_en, data = movies)
boxplot(popularity ~ top_genere, data = movies)
boxplot(popularity ~ Directing , data = movies)
boxplot(popularity ~ release_year , data = movies, outline=FALSE)
boxplot(popularity ~ release_month , data = movies, outline=FALSE)

### check tagline and original lang english witout outliers
boxplot(popularity ~ tagline, data = movies, outline=FALSE)
boxplot(popularity ~ original_lang_en, data = movies, outline=FALSE)

In [ ]:
options(repr.plot.width = 27, repr.plot.height = 8)
boxplot(popularity ~ top_genere, data = movies, outline=FALSE)
options(repr.plot.width = 8, repr.plot.height = 8)

POPULARITY is related to tagline,Original language EN, female/male actor ratio and genere
Found anomalies in release year: 2017 has very high values
genere - Docomentary has low popularity

## Revenue Rank

In [ ]:
summary(movies$revenue_rank)
ggplot(data=movies) +
    geom_density(aes(revenue_rank))
table(movies$revenue_rank)
plot(table(movies$revenue_rank))

In [ ]:
ggplot(data=movies) +
    geom_point(aes(x=id, y=revenue_rank))

Anomalies in movies_id > 3000, probably since these movies have NULL revenue values

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
plot(revenue_rank~log(revenue) , data=movies)
plot(revenue_rank~log(budget) , data=movies)
plot(revenue_rank~runtime, data=movies)
plot(revenue_rank~popularity , data=movies)
plot(revenue_rank~keyword_cnt , data=movies)
plot(revenue_rank ~ actors_female_count, data=movies)
plot(revenue_rank~actors_male_count , data=movies)
plot(revenue_rank~actors_female_ratio , data=movies)

In [ ]:
boxplot(revenue_rank ~ actors_female_count, data=movies)
## No relationship between the variables

In [ ]:
### analize categorial variables
boxplot(revenue_rank ~ popularity_cat, data = movies)
boxplot(revenue_rank ~ runtime_cat, data = movies)
boxplot(revenue_rank ~ homepage, data = movies)
boxplot(revenue_rank ~ tagline, data = movies)
boxplot(revenue_rank ~ in_collection, data = movies)
boxplot(revenue_rank ~ original_lang_en, data = movies)
boxplot(revenue_rank ~ Directing , data = movies)
boxplot(revenue_rank ~ release_year , data = movies, outline=FALSE)
boxplot(revenue_rank ~ release_month , data = movies, outline=FALSE)

In [ ]:
options(repr.plot.width = 27, repr.plot.height = 8)
boxplot(revenue_rank ~ top_genere, data = movies)
options(repr.plot.width = 8, repr.plot.height = 8)

In [ ]:
# filter only high rank movies (1st quartile) and analyze again
high_rev_rank <- movies %>% group_by(release_year) %>% filter(revenue_rank < 28)
table(high_rev_rank$revenue_rank)
plot(table(high_rev_rank$revenue_rank))

In [ ]:
ggplot(data=high_rev_rank) +
    geom_density(aes(revenue_rank))
ggplot(data=high_rev_rank) +
    geom_point(aes(x=id, y=revenue_rank))

In [ ]:
plot(log(revenue) ~ revenue_rank, data=high_rev_rank)
plot(log(budget) ~ revenue_rank, data=high_rev_rank)
plot(runtime ~ revenue_rank, data=high_rev_rank)
plot(popularity ~ revenue_rank, data=high_rev_rank)
plot(keyword_cnt ~ revenue_rank, data=high_rev_rank)
plot(revenue_rank ~ actors_female_count, data=high_rev_rank)
plot(actors_male_count ~ revenue_rank, data=high_rev_rank)
plot(actors_female_ratio ~ revenue_rank, data=high_rev_rank)
boxplot(revenue_rank ~ actors_female_count, data=high_rev_rank)

In [ ]:
boxplot(revenue_rank ~ popularity_cat, data = high_rev_rank)
boxplot(revenue_rank ~ runtime_cat, data = high_rev_rank)
boxplot(revenue_rank ~ homepage, data = high_rev_rank)
boxplot(revenue_rank ~ tagline, data = high_rev_rank)
boxplot(revenue_rank ~ in_collection, data = high_rev_rank)
boxplot(revenue_rank ~ original_lang_en, data = high_rev_rank)
boxplot(revenue_rank ~ top_genere, data = high_rev_rank)
boxplot(revenue_rank ~ Directing , data = high_rev_rank)
boxplot(revenue_rank ~ release_year , data = high_rev_rank, outline=FALSE)
boxplot(revenue_rank ~ release_month , data = high_rev_rank, outline=FALSE)

In [ ]:
options(repr.plot.width = 27, repr.plot.height = 8)
boxplot(revenue_rank ~ top_genere, data = high_rev_rank)
options(repr.plot.width = 8, repr.plot.height = 8)

REVENUE RANK - Found relationship between revenue rank to homepage, collection, original lang EN and genre 'Western'
relationship is weaker (genre,homepage) when looking at high rank movies only

## Runtime

In [ ]:
summary(movies$runtime)
plot(runtime~id,data=movies)
table(movies$runtime)
ggplot(data=movies) +
    geom_density(aes(runtime))
### normal distribution

In [ ]:
plot(runtime~log(revenue), data=movies)
plot(runtime~log(budget) , data=movies)
plot(sqrt(runtime)~ sqrt(popularity), data=movies)
plot(runtime~keyword_cnt, data=movies)
plot(runtime~revenue_rank , data=movies)
plot(runtime~actors_male_count, data=movies)
plot(runtime~actors_female_ratio , data=movies)

In [ ]:
### analize categorial variables
options(repr.plot.width = 8, repr.plot.height = 8)
boxplot(runtime ~ popularity_cat, data = movies)
boxplot(runtime ~ runtime_cat, data = movies)
boxplot(runtime ~ homepage, data = movies)
boxplot(runtime ~ tagline, data = movies)
boxplot(runtime ~ in_collection, data = movies)
boxplot(runtime ~ original_lang_en, data = movies)
boxplot(runtime ~ top_genere, data = movies)
boxplot(runtime ~ Directing , data = movies)
boxplot(runtime ~ release_year , data = movies, outline=FALSE)
boxplot(runtime ~ release_month , data = movies, outline=FALSE)
boxplot(runtime ~ top_genere , data = movies, outline=FALSE)

RUNTIME - No relationship with other variables

## Budget

In [ ]:
summary(movies$budget)
ggplot(data=movies) +
    geom_density(aes(sqrt(budget+0.1)))
ggplot(data=movies) +
    geom_point(aes(x=id, y=budget))

In [ ]:
plot(revenue ~ budget, data=movies)
plot(budget~revenue_rank, data=movies)
plot(budget~runtime, data=movies)
plot(budget~popularity , data=movies)
plot(budget~keyword_cnt, data=movies)
plot(budget~actors_female_ratio, data=movies)

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
boxplot(budget ~ popularity_cat, data = movies, outline=FALSE)
boxplot(budget ~ runtime_cat, data = movies)
boxplot(budget ~ homepage, data = movies)
boxplot(budget ~ tagline, data = movies)
boxplot(budget ~ in_collection, data = movies)
boxplot(sqrt(budget) ~ original_lang_en, data = movies)
boxplot(budget ~ Directing , data = movies)
boxplot(budget ~ release_year , data = movies, outline=FALSE)
boxplot(budget ~ release_month , data = movies, outline=FALSE)

In [ ]:
options(repr.plot.width = 27, repr.plot.height = 8)
boxplot(budget ~ top_genere , data = movies, outline=FALSE)

BUDGET - is in correlation with revenue,popularity_cat, runtime_cat(weak), original_lang_EN, Top_genre - Docometary and foreign movies have significant low budget

## actors_female_count

In [ ]:
summary(movies$actors_female_count)
table(movies$actors_female_count)
barplot(table(movies$actors_female_count))

In [ ]:
ggplot(movies, aes(x = actors_female_count, y = log(budget)))+
    geom_point()
boxplot(log(revenue)~actors_female_count, data=movies)
boxplot(revenue_rank~actors_female_count, data=movies)
ggplot(movies, aes(x = actors_female_count, y = runtime))+
    geom_point()
boxplot(sqrt(popularity)~actors_female_count , data=movies,outline=FALSE)
boxplot(keyword_cnt~actors_female_count, data=movies,outline=FALSE)
plot(actors_male_count~actors_female_count, data=movies)
plot(actors_female_ratio~actors_female_count, data=movies)
ggplot(movies, aes(x = release_year , y = actors_female_count))+
    geom_point()


In [ ]:
options(repr.plot.width = 16, repr.plot.height = 8)
library(gmodels)

ggplot(data=movies) +
    geom_bar(aes(actors_female_count, group=popularity_cat, color=popularity_cat),position = 'dodge')
ggplot(data=movies) +
    geom_bar(aes(actors_female_count, group=runtime_cat, color=runtime_cat),position = 'dodge')
ggplot(data=movies) +
    geom_density(aes(actors_female_count, group=homepage, color=homepage))
ggplot(data=movies) +
    geom_bar(aes(actors_female_count, group=tagline, color=tagline),position = 'dodge')
ggplot(data=movies) +
    geom_bar(aes(actors_female_count, group=original_lang_en, color=original_lang_en),position = 'dodge')

Variable: FEMALE_ACTOR_COUNT is related to many variables: budget,revenue,Popularity,keyword_cnt,release year and male_Actor_count

### Male_actor_count

In [ ]:
summary(movies$actors_male_count)
table(movies$actors_male_count)
barplot(table(movies$actors_male_count))
ggplot(movies, aes(x = id , y = actors_male_count))+
    geom_point()

In [ ]:
ggplot(movies, aes(x = actors_male_count, y = log(budget)))+
    geom_point()
boxplot(log(revenue)~actors_male_count, data=movies)
boxplot(revenue_rank~actors_male_count, data=movies)
ggplot(movies, aes(x = actors_male_count, y = runtime))+
    geom_point()
boxplot(sqrt(popularity)~actors_male_count , data=movies,outline=FALSE)
boxplot(keyword_cnt~actors_male_count, data=movies,outline=FALSE)
plot(actors_female_count~actors_male_count, data=movies)
plot(actors_female_ratio~actors_male_count, data=movies)
plot(release_month~actors_male_count, data=movies)

In [ ]:
ggplot(movies, aes(x = actors_male_count, y = budget))+
    geom_point()

ggplot(data=movies) +
    geom_bar(aes(actors_male_count, group=popularity_cat, color=popularity_cat),position = 'dodge')
ggplot(data=movies) +
    geom_bar(aes(actors_male_count, group=runtime_cat, color=runtime_cat),position = 'dodge')
ggplot(data=movies) +
    geom_density(aes(actors_male_count, group=homepage, color=homepage))
ggplot(data=movies) +
    geom_bar(aes(actors_male_count, group=tagline, color=tagline),position = 'dodge')
ggplot(data=movies) +
    geom_bar(aes(actors_male_count, group=original_lang_en, color=original_lang_en),position = 'dodge')

In [ ]:
str(movies$actors_male_count)
ggplot(movies, aes(x = release_year , y = actors_male_count))+
    geom_point()

Variable: MALE_ACTOR_COUNT is related to female_Actor_count,release year and weak relation to keyword_cnt

### Genre

In [ ]:
options(repr.plot.width = 26, repr.plot.height = 8)
ggplot(data=movies) +
    geom_bar(aes(top_genere))
table(movies$top_genere)

In [ ]:
options(repr.plot.width = 26, repr.plot.height = 8)
plot(revenue~top_genere , data=movies,outline=FALSE)
plot(budget~top_genere , data=movies,outline=FALSE)
plot(runtime~top_genere, data=movies)
plot(popularity~top_genere , data=movies,outline=FALSE)
plot(keyword_cnt~top_genere , data=movies,outline=FALSE)
plot(actors_female_count~top_genere, data=movies,outline=FALSE)
plot(actors_male_count~top_genere , data=movies,outline=FALSE)
plot(actors_female_ratio~top_genere , data=movies,outline=FALSE)

ggplot(data=movies) +
    geom_bar(aes(top_genere, group=runtime_cat, color=runtime_cat),position = 'dodge')
ggplot(data=movies) +
    geom_bar(aes(top_genere, group=homepage, color=homepage),position = 'dodge')
ggplot(data=movies) +
    geom_bar(aes(top_genere, group=tagline, color=tagline),position = 'dodge')

options(repr.plot.width = 8, repr.plot.height = 8)

GENRE is related to budget and revenue. Further more- Docomentry genere is related to Low popularity and low number of female actors

### Languages
original_lang_en

In [ ]:
table(movies$original_lang_en)
options(repr.plot.width = 8, repr.plot.height = 8)
ggplot(data=movies) +
    geom_density(aes(release_year, group=original_lang_en, color=original_lang_en))

In [ ]:
#show distribution for all languages which are not emglish as original language
movie_lang <- movies %>% filter(original_lang_en == 0)%>% select(id,en,fr,ru,es,de,it,ja)
movies2 <- movies %>% filter(original_lang_en == '0')# %>% select(id,original_title,es,de,ru,fr,ja,it,en)

movie_lang$en <- as.numeric(as.character(movie_lang$en))
movie_lang$es <- as.numeric(as.character(movie_lang$es))
movie_lang$fr <- as.numeric(as.character(movie_lang$fr))
movie_lang$de <- as.numeric(as.character(movie_lang$de))
movie_lang$ru <- as.numeric(as.character(movie_lang$ru))
movie_lang$it <- as.numeric(as.character(movie_lang$it))
movie_lang$ja <- as.numeric(as.character(movie_lang$ja))
movie_lang <- movie_lang %>% mutate(total_en_not_original = rowSums(movie_lang[,2:8]))

movie_lang <- movie_lang %>% mutate(across(everything(), ~ ifelse(is.na(.), 0, .)))
total_cnt_lang <- apply(movie_lang[,2:9],2,sum)
barplot(total_cnt_lang[1:7])

English and French are the most common languages for films that weren't in English as original lang.

### Avg_Popularity_5Y_Actor0, Avg_Popularity_5Y_Actor1

In [ ]:
summary(movies$Avg_Popularity_5Y_Actor0)
summary(movies$Avg_Popularity_5Y_Actor1)
ggplot(data=movies) +
    geom_density(aes(log(Avg_Popularity_5Y_Actor0+1)))
ggplot(data=movies) +
    geom_density(aes(log(Avg_Popularity_5Y_Actor1+1)))

In [ ]:
for (v in names(movies[,2:length(movies)])) {
    if(is.numeric(movies[[v]])==T & v != "Avg_Popularity_5Y_Actor0") {
        plot(log(movies$Avg_Popularity_5Y_Actor0+1) ~ movies[[v]], main=v)
    }
}

In [ ]:
for (v in names(movies[,2:length(movies)])) {
    if(is.factor(movies[[v]])==T) {
        suppressWarnings(plot(log(movies$Avg_Popularity_5Y_Actor0+1) ~ movies[[v]], main=v, outline=F))
    }
}

# Outliers

שאלה 3.1
התפלגות של המשתנה עם ובלי ערכי הקיצון

In [ ]:
numlst <- numlst[2:length(numlst)] # remove id variable

In [ ]:
# Create a categorial variable for Revenue
movies$revenue_cat <- ifelse(movies$revenue>16810000,1,0) # median of revenue is 16,810,000
summary(movies$revenue_cat)

In [ ]:
outlierMatrix <- function(data,threshold=1.5) {
    vn <- names(data)
    outdata <- data.frame(row1=1:nrow(data))
    for(v in vn) {
        if(is.numeric(data[[v]])) {
            outlow <- quantile(data[[v]],probs = 0.25,na.rm = T) 
            outhigh <- quantile(data[[v]],probs = 0.75, na.rm = T)
            irq_level <- (outhigh - outlow) * threshold
            outlow <- outlow - irq_level
            outhigh <- outhigh +  irq_level
            mv <- ifelse(data[[v]] < outlow | data[[v]] > outhigh, 1, 0)
            outdata[v] <- mv
        } else {
            mv <- rep(0,nrow(data))
        }
    }
    outdata$row1 <- NULL
    return(outdata)
}

In [ ]:
m_out <- outlierMatrix(movies)
summary(m_out)

שאלה 3.1.1 
בדיקת השפעת נתוני הקיצון על התפלגות המשתנים

In [ ]:
# Check for Distribution change with/without outliers values
res1 <- NULL
for (n in numlst) {
    out <- movies[[n]]
    non <- movies[[n]][which(m_out[[n]]==0)]
    outnum <- length(out) - length(non)
    pval <- suppressWarnings(ks.test(out,non)$p.value)
    res1 <- rbind(res1, cbind(var=n, outliers_cnt=outnum, distributation_chg=ifelse(pval<0.05,'+','-')))
}
res1 

In [ ]:
# Test difference in distribution with/without missing valus for all vars
options(repr.plot.width = 10, repr.plot.height = 8)
for (n in numlst){
    if (n %in% c('popularity','budget')){
        hist(log(movies[[n]]+1), breaks=30, col=rgb(1,0,0,0.5),main=n, xlab=n, ylab="frequency")
        hist(log(movies[[n]][which(m_out[[n]]==0)]+1), breaks=30, col=rgb(0,0,1,0.5), add=T,xlab= c(n,"w/o outliers"))
        legend("topright", legend=c("with Outliers","without Outliers"), col=c(rgb(1,0,0,0.5), 
         rgb(0,0,1,0.5)), pt.cex=2, pch=15 )
        }
    else {
        hist(movies[[n]], breaks=30, col=rgb(1,0,0,0.5),main=n, xlab=n, ylab="frequency")
        hist(movies[[n]][which(m_out[[n]]==0)], breaks=30, col=rgb(0,0,1,0.5), add=T,xlab= c(n,"w/o outliers"))
        legend("topright", legend=c("with Outliers","without Outliers"), col=c(rgb(1,0,0,0.5), 
         rgb(0,0,1,0.5)), pt.cex=2, pch=15 )
    }
}

שאלה 3.1.2
השוואת של כל המשתנים מול משתנה המטרה עם/ללא נתוני קיצון 

In [ ]:
#check correlation with revenue with/without outliers for all numeric vars
options(repr.plot.width = 10, repr.plot.height = 10)
for (n in numlst) {
    plot(movies[[n]], movies$revenue, main=n, xlab=n,col=rgb(1,0,0,0.5), ylab='Revenue')
    points(movies[[n]][which(m_out[[n]]==0)], movies$revenue[which(m_out[[n]]==0)],col=rgb(0,1,0,0.5),ylab='Revenue', pch=10)
    legend("topright", legend=c("with Outliers","without Outliers"), col=c(rgb(1,0,0,0.5), 
     rgb(0,1,0,0.5)), pt.cex=2, pch=15 )
}

In [ ]:
# Check for Correlation change with/without outliers values
library(cocor)
res2 <- NULL
movies_non <- NULL
for (n in numlst[c(1:18,20)]) { #problem with Lighting variable- discard it from the test
    out <- movies[[n]]
    non <- movies[[n]][which(m_out[[n]]==0)]
    revenue_out <- as.numeric(movies$revenue_cat)
    revenue_non <- revenue_out[which(m_out[n]==0)]
    outdf <- data.frame(x_out=out,y_out=revenue_out)
    nondf <- data.frame(x_non=non,y_non=revenue_non)
    movies_non <- suppressWarnings(cbind(movies_non,non))
    
    cr <- suppressWarnings(cocor(~ x_out+y_out | x_non+y_non, data=list(outdf,nondf)))
    pval <- cr@fisher1925$p.value
    res2 <- rbind(res2, cbind(var=n, correlation_changed=ifelse(pval<0.05,'+','-')))
}
res2

In [ ]:
library(tidyr)
res1 <- data.frame(res1)
res2 <- data.frame(res2)
res3 <- plyr::join(res1,res2,by='var',type= 'inner')
res3$drop <- ifelse(res3$distributation_chg == '+' & res3$correlation_changed == '+' , 'NO','YES')
res3

Remove outliers
שאלה 3.1.3

In [ ]:
remove_outliers <- function(x, na.rm = TRUE, ...) {
  qnt <- quantile(x, probs=c(.25, .75), na.rm = na.rm, ...)
  H <- 1.5 * IQR(x, na.rm = na.rm)
  y <- x
  y[x < (qnt[1] - H)] <- NA
  y[x > (qnt[2] + H)] <- NA
  y
}

In [ ]:
# Remove outliers
for (row in 1:nrow(res3)) {
    x <- res3[row,'drop']
    y <- res3[row,'var']
    if(x == 'YES') {      
        movies[[y]] <- remove_outliers( movies[[y]])
        print(y)
        }
}

Keep outliers for Popularity,Budget,Avg_Popularity_5Y_Actor0, Crew departments (Art,Sound,Crew,Costume,Camera)

# Missing Values

Few variables have missing values
I'll look for correlation with other varriables for explaination

In [ ]:
df_na <- df_na[,mm$missingness$var]
library(corrgram)
corrgram(df_na, order=TRUE, lower.panel=panel.shade,
  upper.panel=panel.pie, text.panel=panel.txt,
  main="Missingness correlation")

The variables: Actor male/female count have correlation with all the variables of the crew departments.
Avg_popularity actor0 is in correlation with male/female count
All other variables have NO apperent correlation with other vars

In [ ]:
 sapply(movies, function(x) length(unique(x)))

In [ ]:
### test the missing mechanism using glm
nm <- setdiff(names(movies),c("id","original_title","revenue","revenue_cat"))

res <- NULL

for (m in mm$missingness$var) {
    ## change the current tested variable with the missing indicator for this variable
    ff <- movies[,nm[c(1:4,5:6,9:26:37)]]#,7:24,37)]]# some variable cannot be used in the glm model 
                                        # ERROR "contrasts can be applied only to factors with 2 or more levels"
    ff[[m]] <- df_na[[m]]
    mod <- suppressWarnings(glm(ff[[m]] ~.,data=ff,family="binomial"))
    sm <- summary(mod)
    if(is.null(sm)==F) {
      sm2 <- data.frame(var=row.names(sm$coefficients),pvalue=sm$coefficients[,4])
      res <- rbind(res, cbind(m,sm2))
    } else {
      print(sm)
    }
}

row.names(res) <- NULL
res

In [ ]:
getMissingness <- function (data, getRows = FALSE) {
    require(dplyr)
    l <- nrow(data)
    vn <- names(data)
    nadf <- data
    cnt <- NULL
    miss <- function(x) return(sum(is.na(x)))
    for (n in vn) {
        nadf[[n]] <- ifelse(is.na(nadf[[n]]) == T, 1, 0)
        cnt <- rbind(cnt, data.frame(n, sum(nadf[[n]])))
    }
    names(cnt) <- c("var", "na.count")
    cnt$rate <- round((cnt$na.count/nrow(nadf)) * 100, 1)
    nadf$na.cnt <- 0
    nadf$na.cnt <- rowSums(nadf)
    cnt <- cnt %>% dplyr::arrange(desc(na.count)) %>% dplyr::filter(na.count > 
        0)
    totmiss <- nadf %>% dplyr::filter(na.cnt == 0) %>% dplyr::tally()
    idx <- NULL
    msg <- (paste("This dataset has ", as.character(totmiss), 
        " (", as.character(round(totmiss/nrow(data) * 100, 1)), 
        "%)", " complete rows. Original data has ", nrow(data), 
        " rows.", sep = ""))
    if (getRows == TRUE & totmiss != 0) {
        nadf$rn <- seq_len(nrow(data))
        idx <- nadf %>% dplyr::filter(na.cnt == 0) %>% dplyr::select(rn)
    }
    print(list(head(cnt, n = 10), msg))
    return(list(missingness = cnt, message = msg, rows = idx$rn))
}

In [ ]:
res <- data.frame(res)
row.names(res) <- NULL
res %>% filter(pvalue<0.05)

None of the variables can explain the missing values

In [ ]:
mm <- getMissingness(data = movies)

In [ ]:
# check lines with large number of missing values
dfna_sum <- df_na
dfna_sum$pct <- rowSums(df_na)/ncol(df_na)
dfna_sum$var <- col(df_na)
dfna_sum %>% group_by(pct) %>% tally

drop rows which have more than 50% missing values

In [ ]:
movies$drop <- ifelse(dfna_sum$pct >= 0.5,1,0)
movies <- movies %>% filter(movies$drop==0)
movies$drop <- NULL
df_na <- missingMatrix(movies)

In [ ]:
# after dropping lines with high % of missing values
dfna_sum <- df_na
dfna_sum$pct <- rowSums(df_na)/ncol(df_na)
dfna_sum %>% group_by(pct) %>% tally

In [ ]:
# Missing heatmap after removing the lines
library(naniar)
options(repr.plot.width = 20, repr.plot.height = 12)
vis_miss(movies[1:40],warn_large_data=F)

In [ ]:
library(gridExtra)
options(repr.plot.width = 12, repr.plot.height = 12)
for(m in mm$missingness$var) {
    p <- list()
    i <- 1
    for (v in numlst[2:length(numlst)]) {
        if (v != m) {
            ff <- data.frame(v=movies[[v]],m=factor(df_na[[m]]))
            p[[i]] <- suppressMessages(ggplot(data=ff,aes(x=v,group=m,color=m)) +
                geom_density()+labs(title=paste(v, "(miss=", m,")"),x=v))
            i <- i+1
        }
    }
    do.call(grid.arrange,p)
}

The variables: revenue_rank and female_actor_ratio seem to have different distribution when removing missing values.
I'll check for significent difference using Kolmogorov-Smirnov

In [ ]:
res4 <- NULL
for (m in mm$missingness$var) {
    p <- list()
    for (n in numlst) {
        if (n != m) {
            miss <- movies[[n]]
            non <- movies[[n]][which(df_na[[n]]==0)]
            missnum <- length(miss) - length(non)
            pval <- suppressWarnings(ks.test(miss, non)$p.value)
            res4 <- rbind(res4, cbind(var=n,missing=m, miss_cnt=missnum, distribution_changed=ifelse(pval<0.05,1,0)))
            }
    }
}
res4

In [ ]:
res4 <- data.frame(res4)
res4 %>% filter(res4$distribution_changed=='1')

I haven't found any variable that can explain the missing values thus I can assume at least MAR

In [ ]:
#install.packages("VIM")
library(mice)
library(VIM)
missing_plot <- aggr(movies, col=c('navyblue','yellow'),
                    numbers=TRUE, sortVars=TRUE,
                    labels=names(movies), cex.axis=.7,
                    gap=3, ylab=c("Missing data","Pattern"))

I have found that all the missing values are MAR and I'll use KNN algorithem to fill the NA's

In [ ]:
library(colorspace)
library(VIM)
df <- movies[c(1,3:4,6:(length(movies)-1))]
movies_imp <- kNN(df,k=5)

# חלק 4

In [ ]:
movies_imp$revenue <- movies$revenue
colnames(movies_imp)
indx <- grepl('imp', colnames(movies_imp))
movies_imp <- movies_imp[!indx]

In [ ]:
tab1 <- suppressWarnings(mechkar::Table1(data=movies_imp[,3:ncol(movies_imp)]))

In [ ]:
suppressWarnings(mechkar::exploreData(data=movies_imp,y="revenue",dir="/Users/maozdotan/Documents/DS"))

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
for (n in names(movies_imp[3:length(movies_imp)])) {
    suppressWarnings(
        if (n %in% numlst) {hist(movies_imp[[n]],xlab=n, main=n)}
    else {plot(movies_imp[[n]],main=n)})
}

In [ ]:
library(MESS)
####
options(repr.plot.width = 20, repr.plot.height = 20)
pairs(movies_imp[,numlst], 
      upper.panel = panel.smooth,
      diag.panel = panel.hist, 
      lower.panel = NULL,
      cex=0.8)

In [ ]:
options(repr.plot.width = 25, repr.plot.height = 25)
library(corrgram)
dfcormat <- cor(movies_imp[,c(numlst,'revenue')], use = "pairwise.complete.obs", method = "spearman")
corrgram::corrgram(dfcormat)

In [ ]:
library(rcompanion)
library(lsr)
res <- correlate(movies_imp, corr.method = "spearman")
res

In [ ]:
options(repr.plot.width = 5, repr.plot.height = 5)
summary(movies_imp$revenue)
hist(movies_imp$revenue, main='Revenue')
hist(log(movies_imp$revenue+1) ,main='Revenue log')
plot(log(movies_imp$budget+1) ~ log(movies$revenue+1), main='Revenue vs. Budget')
plot(log(movies_imp$Avg_Popularity_5Y_Actor0+1) ~ log(movies_imp$revenue+1), main='Revenue vs. Avg_Popularity_5Y_Actor0')
plot(log(movies_imp$Avg_Popularity_5Y_Actor1+1) ~ log(movies_imp$revenue+1),main='Revenue vs. Avg_Popularity_5Y_Actor1')

for (n in numlst){
    if(!n %in% c('budget','Avg_Popularity_5Y_Actor0','Avg_Popularity_5Y_Actor1')) {
        plot(log(movies_imp$revenue+1) ~ movies_imp[[n]], main= n)
    }
}

In [ ]:
#categoial variables
options(repr.plot.width = 8, repr.plot.height = 8)
for (v in names(movies_imp[,3:41])) { 
    if(is.factor(movies_imp[[v]])==T) {
        suppressWarnings(boxplot(log(movies_imp$revenue+1) ~ movies_imp[[v]], main=v, outline=F))
    }
}

The outcome has high positive corraletion to :
(both with outliers retained)
1. budget
2. popularity

It is associated with:
1. average popularity of main actors (order 0 - positive up to popularity 40 and then negative)
2. female/male actor ratio - strong correlation up to ~0.5 and then no correlation
3. original language english
4. homepage available
5. tagline
6. top genre

variable correlation with:
1. keyword count - positive up to 50 and then negative
2. in collection - strong positive up to 7 and then negative
3. rutime - strong positive bteween 100-200 min and negative beyond 200 min

In [ ]:
m_out_imp <- outlierMatrix(movies_imp)

In [ ]:
# Check for Distribution change with/without outliers values
res_imp <- NULL
for (n in numlst) {
    out <- movies_imp[[n]]
    non <- movies_imp[[n]][which(m_out_imp[[n]]==0)]
    outnum <- length(out) - length(non)
    #pval <- suppressWarnings(ks.test(out,non)$p.value)
    res_imp <- rbind(res_imp, cbind(var=n, outliers_cnt=outnum))#, distributation_chg=ifelse(pval<0.05,'+','-')))
}
res_imp

Popularity and Avg_Popularity_5Y_Actor0 have large outliers that need to be evaluated

In [ ]:
options(repr.plot.width = 8, repr.plot.height = 8)
boxplot(movies_imp$popularity,main ="popularity")
boxplot(movies_imp$Avg_Popularity_5Y_Actor0,main="Avg_Popularity_5Y_Actor0")
movies_tmp <- movies_imp
movies_tmp$popularity <- remove_outliers(movies_tmp$popularity)
qplot(x=popularity, y=revenue, data=movies_tmp,geom='jitter', method='smooth',main="removed outliers")
qplot(x=popularity, y=revenue, data=movies_imp,geom='jitter', method='smooth')
movies_tmp$Avg_Popularity_5Y_Actor0 <- remove_outliers(movies_tmp$Avg_Popularity_5Y_Actor0)
qplot(x=Avg_Popularity_5Y_Actor0, y=revenue, data=movies_tmp,geom='jitter', method='smooth',main="removed outliers")
qplot(x=Avg_Popularity_5Y_Actor0, y=revenue, data=movies_imp,geom='jitter', method='smooth')

In [ ]:
# Test difference in distribution with/without outliers valus for all vars
options(repr.plot.width = 10, repr.plot.height = 8)

for (n in numlst){ #discard id var
    if (n %in% c('popularity','budget')){
        hist(log(movies[[n]]+1), breaks=30, col=rgb(1,0,0,0.5),main=n, xlab=c("log",n), ylab="frequency")
        hist(log(movies[[n]][which(m_out_imp[[n]]==0)]+1), breaks=30, col=rgb(0,0,1,0.5), add=T,xlab= c(n,"w/o outliers"))
        legend("topright", legend=c("with Outliers","without Outliers"), col=c(rgb(1,0,0,0.5), 
         rgb(0,0,1,0.5)), pt.cex=2, pch=15 )
        }
    else {
        hist(movies[[n]], breaks=30, col=rgb(1,0,0,0.5),main=n, xlab=n, ylab="frequency")
        hist(movies[[n]][which(m_out_imp[[n]]==0)], breaks=30, col=rgb(0,0,1,0.5), add=T,xlab= c(n,"w/o outliers"))
        legend("topright", legend=c("with Outliers","without Outliers"), col=c(rgb(1,0,0,0.5), 
         rgb(0,0,1,0.5)), pt.cex=2, pch=15 )
    }
}

In [ ]:
#check correlation with revenue with/without outliers for all numeric vars
options(repr.plot.width = 10, repr.plot.height = 10)
for (n in numlst) {
    plot(movies_imp[[n]], movies_imp$revenue, main=n, xlab=n,col=rgb(1,0,0,0.5), ylab='Revenue')
    points(movies_imp[[n]][which(m_out_imp[[n]]==0)], movies_imp$revenue[which(m_out_imp[[n]]==0)],col=rgb(0,1,0,0.5),ylab='Revenue', pch=10)
    legend("topright", legend=c("with Outliers","without Outliers"), col=c(rgb(1,0,0,0.5), 
     rgb(0,1,0,0.5)), pt.cex=2, pch=15 )
}

In [ ]:
mm1 <- mechkar::getMissingness(data = movies_imp)
mm1$missingness
mm1$missingness$var

In [ ]:
df_na_imp <- missingMatrix(movies)

In [ ]:
library(naniar)
options(repr.plot.width = 20, repr.plot.height = 12)
vis_miss(movies_imp,warn_large_data=F)